# ns-cLBP Cohort Selection

We select a cohort of patients form UCSF's information common's database, identified by unique patient identifiers (PatientDurableKeys) and unique imaging identifiers (AccessionNumbers).

This script is intended to be run on Wynton:
1. Loads EMR tables relevant to patient selection criteria.
2. Identifies patients who meet inclusion criteria.
3. Out of possible matches, identifiers patients who meet exclusion criteria. 
4. Selects relevant patients by identifying the set of PatientDurableKeys that meet all criteria.
5. Save unique patient identifiers (PatientDurableKeys) and unique imaging identifiers (AccessionNumbers)



In [ ]:
import duckdb
import pandas as pd
import os

import pyrootutils
root = pyrootutils.setup_root(
    search_from=os.path.abspath(''),
    indicator=[".git"],
    pythonpath=True, # add root directory to the PYTHONPATH (helps with imports)
    dotenv=True, # load environment variables from .env if exists in root directory
)
from src.utils.query_utils.extractor import Extractor

In [ ]:
# --- LOAD RELATIONAL DATABASE ---
datapath = #TODO 'path_to_your_data/parquet'
data_asset = 'DEID_CDW'

# Enter tables you are planning to query here
imagingfact             = duckdb.read_parquet(os.path.join(datapath,data_asset,'imagingfact','*.parquet'))
medicationorderfact     = duckdb.read_parquet(os.path.join(datapath,data_asset,'medicationorderfact','*.parquet'))
diagnosiseventfact      = duckdb.read_parquet(os.path.join(datapath,data_asset,'diagnosiseventfact','*.parquet'))
diagnosisterminologydim = duckdb.read_parquet(os.path.join(datapath,data_asset,'diagnosisterminologydim','*.parquet'))
procedureeventfact      = duckdb.read_parquet(os.path.join(datapath,data_asset,'procedureeventfact','*.parquet'))
patdurabledim           = duckdb.read_parquet(os.path.join(datapath,data_asset,'patdurabledim','*.parquet'))

# Initialize data extraction tools
check_query_flag = True       #axilluary checks to see if query makes sense
PlumsExtractor = Extractor(num_results_flag=True, display_results_flag=True)

## Cohort Identification

### Inclusion Criteria

In [ ]:
imaging_query = f"""
/*
Created By: Michelle Tong
Description: Select patients with a lumbar spine MRI between 2012-2024.
*/
SELECT DISTINCT
    deidlds, 
    accessionnumber, 
    patientdurablekey, 
    encounterkey, 
    firstprocedurename, 
    examstartdatekey,
    examstartdatekeyvalue -- to compare with bday
FROM imagingfact 
  WHERE canceled=0
  AND examstartdatekey >= 20120601
  AND 
    (
    UPPER(firstprocedurename) LIKE '%LUMB%' -- LUMBAR
    OR UPPER(firstprocedurename) LIKE '%L_SPINE%' -- L-SPINE & L SPINE, FETAL SPINE, TOTAL SPINE
    )
    AND UPPER(firstprocedurename) LIKE '%MR%'
    AND UPPER(firstprocedurename) LIKE '%SPINE%'

    AND UPPER(firstprocedurename) NOT LIKE '%CERV%' -- CERVICAL
    AND UPPER(firstprocedurename) NOT LIKE '%THOR%' -- THORACIC
    AND UPPER(firstprocedurename) NOT LIKE '%FETAL%'
    AND UPPER(firstprocedurename) NOT LIKE '%TOTAL%'
    AND UPPER(firstprocedurename) NOT LIKE '%OUTSIDE%' -- exam at occured outside UCSF
    AND UPPER(firstprocedurename) NOT LIKE '%LIMITED%' -- exclude quick scans
    AND 
    (
    UPPER(firstprocedurename) LIKE 'MR LUMBAR SPINE WITHOUT CONTRAST (O)'
    OR UPPER(firstprocedurename) LIKE 'MR LUMBAR SPINE WITHOUT CONTRAST' 
    )
    AND UPPER(orderingdepartmentname) NOT LIKE '%PED%'
    AND 
    (
    UPPER(orderingdepartmentspecialty) NOT LIKE '%PED%'
    OR UPPER(orderingdepartmentspecialty) LIKE '%ORTHOPED%'
    )
  ORDER BY
    examstartdatekeyvalue,
    firstprocedurename,
    patientdurablekey
"""

tmp_img_df = PlumsExtractor.run_query(imaging_query, runtime_flag=True)

patientdurablekey_workinglist = tmp_img_df['patientdurablekey'].drop_duplicates()
print('total patientdurablekeys: ',len(patientdurablekey_workinglist)) # number of rows in df

tmp_img_df.head()

### Exclusion Criteria

In [ ]:
bday_query = f"""
/*
Description: Select patients who are younger than 18 years old at the time of their first MRI scan.
*/
SELECT 
  i.deidlds, 
  i.accessionnumber, 
  i.patientdurablekey, 
  i.encounterkey, 
  i.firstprocedurename, 
  i.examstartdatekey,
  i.examstartdatekeyvalue,
  p.birthdate,
  DATE_ADD(p.birthdate, INTERVAL 18 YEAR) AS date_18_years
FROM tmp_img_df as i
JOIN patdurabledim as p
ON 
  i.patientdurablekey = p.patientdurablekey
WHERE 
  i.examstartdatekeyvalue < DATE_ADD(p.birthdate, INTERVAL 18 YEAR)
ORDER BY
    birthdate DESC
"""

results_df = PlumsExtractor.run_query(bday_query, runtime_flag=True)

patientdurablekey_bday_excludelist = results_df['patientdurablekey'].drop_duplicates()
print('total patientdurablekeys: ',len(patientdurablekey_bday_excludelist)) # number of rows in df

results_df.head()

In [ ]:
# Exclude Surgery and Injections from Procedures
surgery_list = ['DISCECTOMY','LAMINECTOMY','LAMINOTOMY','FORAMINOTOMY','FACETECTOMY']
surgery_abbrev_list = ['ALIF','ASF','PSF','OLIF','TLIF','XLIF']

surgery_query_txt = ' OR '.join(['procedurename LIKE \'%'+x+'%\'' for x in surgery_list])
surgery_abbrev_query_txt = ' OR '.join(['UPPER(procedurename) LIKE \'%'+x+'%\'' for x in surgery_abbrev_list])

surgeryQuery = f'''
/*
Description: Select patients with fusion, surgeries, injections, facet and nerve blocks, radiofrequency ablation from procedures
*/

SELECT DISTINCT
  patientdurablekey,
  procedurekey,
  procedurename, 
  procedurecode,
  procedurecategory,
  procedurestartdatekey,
  procedureenddatekey,
  CASE
      WHEN UPPER(procedurename) LIKE '%INJECTION%' THEN 'INJECTION'
      WHEN UPPER(procedurename) LIKE '%BLOCK%' THEN 'BLOCK'
      WHEN UPPER(procedurename) LIKE '%ABLATION%' THEN 'RADIOFREQUENCY ABLATION'
      WHEN UPPER(procedurename) LIKE '%ARTHRODESIS%' THEN 'FUSION'
      WHEN UPPER(procedurename) LIKE '%FUSION%' THEN 'FUSION'
      WHEN UPPER(procedurename) LIKE '%ALIF%' THEN 'FUSION'
      WHEN UPPER(procedurename) LIKE '%ASF%' THEN 'FUSION'
      WHEN UPPER(procedurename) LIKE '%PSF%' THEN 'FUSION'
      WHEN UPPER(procedurename) LIKE '%OLIF%' THEN 'FUSION'
      WHEN UPPER(procedurename) LIKE '%TLIF%' THEN 'FUSION'
      WHEN UPPER(procedurename) LIKE '%XLIF%' THEN 'FUSION'
      WHEN UPPER(procedurename) LIKE '%SURGERY%' THEN 'SURGERY'
      WHEN UPPER(procedurename) LIKE '%OSTEOTOMY%' THEN 'OSTEOTOMY'
      ELSE 'SURGERY'
      END AS proceduretype
FROM 
  procedureeventfact
  WHERE 
    patientdurablekey IN {tuple(patientdurablekey_workinglist)}
    -- AND procedurestartdatekey > 0 -- 5/24/2024 excluding this line, difference of ~2400 patients
    AND
    (
        --FOR surgery (specific names)
        {surgery_query_txt}
        OR 
        --FOR fusions (full name)
        (
            (
            UPPER(procedurename) LIKE '%FUSION%'
            OR UPPER(procedurename) LIKE '%ARTHRODESIS%'
            )
            AND
            (
            UPPER(procedurename) LIKE '%SPINE%'
            OR UPPER(procedurename) LIKE '%INTERBODY%'
            OR UPPER(procedurename) LIKE '%LUMB%' -- LUMBAR
            )
        )
        OR
        --FOR fusions (abbreviations)
        (
            (
            {surgery_abbrev_query_txt}
            )
            AND
            (
            UPPER(procedurename) LIKE '%SPINE%'
            OR UPPER(procedurename) LIKE '%INTERBODY%'
            OR UPPER(procedurename) LIKE '%LUMBAR%' -- LUMBAR
            OR UPPER(procedurename) LIKE '%FUSION%'
            )
        )
        OR
        --FOR surgeries
        (
            (
            UPPER(procedurename) LIKE '%SURGERY%'
            OR UPPER(procedurename) LIKE '%OSTEOTOMY%'
            -- OR UPPER(procedurename) LIKE '%MIS%' -- currently this describes TLIF which is explicitly queried elsewhere
            )
            AND
            (
            UPPER(procedurename) LIKE '%SPINE%'
            OR UPPER(procedurename) LIKE '%DISC %' -- not DISCharge, lumbar/cervical/thoracic disc surgery
            OR UPPER(procedurename) like '%LUMB%' -- LUMBAR
            )
        )
        OR
        -- FOR injection
        (
            (
            UPPER(procedurename) LIKE '%INJECTION%' -- includes epidural injections
            OR UPPER(procedurename) LIKE '%BLOCK%'
            OR  (UPPER(procedurename) LIKE '%ABLATION%'
                AND UPPER(procedurename) LIKE '%RADIOFREQUENCY%'
                )
            )
            AND 
            (
            UPPER(procedurename) LIKE '%LUMB%'
            )
        )
    )
ORDER BY
    patientdurablekey,
    procedurestartdatekey
    
'''

# Run query and update relevant keys
results_df = PlumsExtractor.run_query(surgeryQuery, runtime_flag=True)

patientdurablekey_procedure_excludelist = results_df['patientdurablekey'].drop_duplicates()
print('total patientdurablekeys: ',len(patientdurablekey_procedure_excludelist)) # number of rows in df

# #Check whether query makes sense
# if check_query_flag==True:
#     PlumsExtractor.col_to_list(results_df,'patientdurablekey')
#     PlumsExtractor.col_to_list(results_df,'encounterkey')
#     PlumsExtractor.col_to_list(results_df,'procedurename')
    
results_df.head()

In [ ]:
# Exclude Confounders from Procedures
exclude_diag_list = ['osteomyelitis','discitis','spondylitis','spondylodiscitis','septic facet joint'] 
exclude_diag_query_txt = ' OR '.join(['LOWER(diagnosisname) LIKE \'%'+x+'%\'' for x in exclude_diag_list])

exclude_diagnosisQuery = f'''
/*
Created By: Michelle Tong
Description: Select patients with conditions likely to cause LBP or meaningful clinical cofounders 
(fracture, lymph, sepsis, stroke, paresis, cancer, infection) from diagnoses
*/

SELECT 
  patientdurablekey, 
  diagnosiseventkey, 
  diagnosiskey, 
  diagnosisname, 
  encounterkey, 
  startdatekey, 
  enddatekey,
  CASE
  WHEN LOWER(diagnosisname) LIKE '%fracture%' THEN 'fracture'
  WHEN LOWER(diagnosisname) LIKE '%failed back%' THEN 'failed back'
  WHEN LOWER(diagnosisname) LIKE '%abscess%' THEN 'abscess'
  WHEN LOWER(diagnosisname) LIKE '%lymph%' THEN 'lymph'
  WHEN LOWER(diagnosisname) LIKE '%stroke%' THEN 'stroke'
  WHEN LOWER(diagnosisname) LIKE '%sepsis%' THEN 'sepsis'
  WHEN LOWER(diagnosisname) LIKE '%paresis%' THEN 'paresis'
  WHEN ({exclude_diag_query_txt}) THEN 'infection'
  WHEN LOWER(diagnosisname) LIKE '%infection%' THEN 'infection'
  WHEN LOWER(diagnosisname) LIKE '%trama%' THEN 'trama'
  WHEN (LOWER(diagnosisname) LIKE '%tumor%' OR LOWER(diagnosisname) LIKE '%metastasis%' OR LOWER(diagnosisname) LIKE '%oncology%') THEN 'cancer'
  ELSE 'category not specified'
  END AS diagnosistype
FROM 
  diagnosiseventfact
  
  WHERE
  patientdurablekey IN {tuple(patientdurablekey_workinglist)}
  AND
  (
    -- diagnosis exclusions
    {exclude_diag_query_txt}
    
    OR LOWER(diagnosisname) LIKE '%fracture%'
    OR LOWER(diagnosisname) LIKE '%failed back%' 
    OR LOWER(diagnosisname) LIKE '%abscess%' 
    
    OR LOWER(diagnosisname) LIKE '%tumor%'
    OR LOWER(diagnosisname) LIKE '%metastasis%'
    OR LOWER(diagnosisname) LIKE '%oncology%'
    OR LOWER(diagnosisname) LIKE '%lymph%'
    OR LOWER(diagnosisname) LIKE '%stroke%'
    OR LOWER(diagnosisname) LIKE '%sepsis%'
    OR LOWER(diagnosisname) LIKE '%paresis%'
    OR LOWER(diagnosisname) LIKE '%spondyloarthr%'
    OR LOWER(diagnosisname) LIKE '%infection%'
    OR LOWER(diagnosisname) LIKE '%trama%'

  )
  
  ORDER BY
    diagnosiskey,
    patientdurablekey
'''


# Run query and update relevant keys
results_df = PlumsExtractor.run_query(exclude_diagnosisQuery, runtime_flag=True)

patientdurablekey_diagnosis_excludelist = results_df['patientdurablekey'].drop_duplicates()
print('total patientdurablekeys: ',len(patientdurablekey_diagnosis_excludelist)) # number of rows in df


results_df.head()

### Patient Selection

In [ ]:
#INCLUSION
print('n=', len(patientdurablekey_workinglist))
print('unique n=', len(set(patientdurablekey_workinglist)))

#EXCLUSION
print(len(patientdurablekey_bday_excludelist))
print(len(patientdurablekey_procedure_excludelist))
print(len(patientdurablekey_diagnosis_excludelist))

#COMBINE EXCLUSION
PID_surg_inj = patientdurablekey_procedure_excludelist.to_list()
print(len(set(PID_surg_inj)))

PID_surg_inj_diag = PID_surg_inj + patientdurablekey_diagnosis_excludelist.to_list()
print(len(set(PID_surg_inj_diag)))

PID_surg_inj_diag_over18yrs = PID_surg_inj_diag + patientdurablekey_bday_excludelist.to_list()
print(len(set(PID_surg_inj_diag_over18yrs)))

#SELECTION
print('eligible n=', len(patientdurablekey_workinglist) - len(set(PID_surg_inj_diag_over18yrs)))
patientdurablekey_selection = set(patientdurablekey_workinglist)-set(PID_surg_inj_diag_over18yrs)

## Save Cohort

In [ ]:
imaging_query = f"""
        SELECT DISTINCT
            deidlds, 
            accessionnumber, 
            patientdurablekey, 
            encounterkey, 
            firstprocedurename, 
            examstartdatekey 
        FROM 
            imagingfact

          WHERE 
          patientdurablekey IN {tuple(patientdurablekey_selection)}
          AND canceled=0
          AND examstartdatekey >= 20120601
          AND 
            (
            firstprocedurename LIKE '%LUMB%' -- LUMBAR
            OR firstprocedurename LIKE '%L_SPINE%' -- L-SPINE & L SPINE, FETAL SPINE, TOTAL SPINE
            )
            AND firstprocedurename LIKE '%MR%'
            AND firstprocedurename LIKE '%SPINE%'

            AND firstprocedurename NOT LIKE '%CERV%' -- CERVICAL
            AND firstprocedurename NOT LIKE '%THOR%' -- THORACIC
            AND firstprocedurename NOT LIKE '%FETAL%'
            AND firstprocedurename NOT LIKE '%TOTAL%'
            AND firstprocedurename NOT LIKE '%OUTSIDE%' -- exam at occured outside UCSF

            AND UPPER(firstprocedurename) NOT LIKE '%LIMITED%' -- exclude quick scans

            -- OPTION 1
            AND (UPPER(firstprocedurename) LIKE 'MR LUMBAR SPINE WITHOUT CONTRAST (O)'
            OR UPPER(firstprocedurename) LIKE 'MR LUMBAR SPINE WITHOUT CONTRAST' )

            -- OPTION 2
            --AND UPPER(firstprocedurename) NOT LIKE 'MR LUMBAR SPINE WITH CONTRAST (O)'
            --AND UPPER(firstprocedurename) NOT LIKE 'MR LUMBAR SPINE WITH CONTRAST'
            
            AND UPPER(orderingdepartmentname) NOT LIKE '%PED%'
            AND (UPPER(orderingdepartmentspecialty) NOT LIKE '%PED%'
            OR UPPER(orderingdepartmentspecialty) LIKE '%ORTHOPED%')
            
          ORDER BY
            firstprocedurename,
            patientdurablekey
"""

results_df = PlumsExtractor.run_query(imaging_query, runtime_flag=True)

patientdurablekey_selected = PlumsExtractor.col_to_list(results_df, 'patientdurablekey', isdistinct=True)
patientdurablekey_selected = PlumsExtractor.remove_invalid(patientdurablekey_selected)
print('total patientdurablekeys: ',len(patientdurablekey_selected)) # number of rows in df

accessionnumber_selected = PlumsExtractor.col_to_list(results_df, 'accessionnumber', isdistinct=True)
accessionnumber_selected = PlumsExtractor.remove_invalid(accessionnumber_selected)
print('total accessionnumbers: ',len(accessionnumber_selected)) # number of rows in df

results_df.head()

In [ ]:
# Load file configuration
from src.utils.file_management.config_loader import load_yaml, process_config_values
from src.utils.file_management.file_manager import FileManager

cohort_cfg_path = #TODO '/path_to_your_project/code/config/datasets/cohort03_MriNoninvasive.yaml'
config = load_yaml(cohort_cfg_path)
config = process_config_values(config)
print(config.keys())

PlumsFiles = FileManager(config.get('file_directory'))

# Define save path
save_path = PlumsFiles.get_datapath('patientdurablekey_csv')
PlumsFiles.save_df_to_csv(pd.DataFrame({'patientdurablekey': patientdurablekey_selected}), save_path)

save_path = PlumsFiles.get_datapath('accessionnumber_csv')
PlumsFiles.save_df_to_csv(pd.DataFrame({'accessionnumber': accessionnumber_selected}), save_path)

#Sample writing to csv
#duckdb.sql().write_csv(PlumsFiles.get_datapath(''))         # Write to a CSV file